In [1]:
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
sys.path.append(str(src_path))

import json
import types
from collections import OrderedDict
from pprint import pprint

import torch
import torch.nn as nn
from dataset import get_new_dataloader
from funcs import get_dataloaders
from models import PreActResNet18, WideResNet
from train import epoch, epoch_test


In [2]:
# constant
CWD_PATH = Path.cwd()
MODEL_DIR_PATH = CWD_PATH.parent / "models/CIFAR10"
LABELS = [
    "model_data-normalization",
    "model_min-max--1-and-1",
    "model_normalization-without-mean",
    "model_normalization-without-std",
    "model_rgb-grayscale",
]

In [3]:
def load_model(
    model_type: str,
    location: str,
    model_normalize: bool,
):
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")

    # load model CIFAR10
    if model_type == "pre-act-18":
        model = PreActResNet18(num_classes=10, normalize=model_normalize)
    elif model_type == "independent":
        model = WideResNet(
            n_classes=10,
            depth=16,  # deep_full for CIFAR10
            widen_factor=1,
            normalize=model_normalize,
            dropRate=0.3,
        )
    else:
        model = WideResNet(
            n_classes=10,
            depth=28,  # deep_full for CIFAR10
            widen_factor=10,
            normalize=model_normalize,
            dropRate=0.3,
        )

    try:
        model = model.to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    except:
        model = nn.DataParallel(model).to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    model.eval()
    return model


In [4]:
model_name__tr_acc = OrderedDict()

# model performance in TR
for label in LABELS:
    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"
    args.experiment = 'diff-normalization'
    args.dataset = 'CIFAR10'
    args.batch_size = 1000
    args.num_workers = 8
    args.normalization_type = label[6:]
    print(args.normalization_type)
    train_loader, test_loader = get_new_dataloader(args=args)

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__tr_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }



/data/weijing/DI-ting_normalization/models/CIFAR10/model_data-normalization/final.pt
data-normalization
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:29<00:00,  1.72it/s]


0.99994
0.9429
/data/weijing/DI-ting_normalization/models/CIFAR10/model_min-max--1-and-1/final.pt
min-max--1-and-1
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.13it/s]


0.99998
0.9443
/data/weijing/DI-ting_normalization/models/CIFAR10/model_normalization-without-mean/final.pt
normalization-without-mean
Files already downloaded and verified
Files already downloaded and verified


...

0.99998
0.9424
/data/weijing/DI-ting_normalization/models/CIFAR10/model_normalization-without-std/final.pt
normalization-without-std
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.12it/s]


1.0
0.9438
/data/weijing/DI-ting_normalization/models/CIFAR10/model_rgb-grayscale/final.pt
rgb-grayscale
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.14it/s]


0.99992
0.9271


In [5]:
pprint(model_name__tr_acc)

# Serializing json
json_object = json.dumps(model_name__tr_acc, indent=2)
 
# Writing to sample.json
with open("tr_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model_data-normalization',
              {'test_acc': 0.9429, 'train_acc': 0.99994}),
             ('model_min-max--1-and-1',
              {'test_acc': 0.9443, 'train_acc': 0.99998}),
             ('model_normalization-without-mean',
              {'test_acc': 0.9424, 'train_acc': 0.99998}),
             ('model_normalization-without-std',
              {'test_acc': 0.9438, 'train_acc': 1.0}),
             ('model_rgb-grayscale',
              {'test_acc': 0.9271, 'train_acc': 0.99992})])


In [8]:
model_name__te_acc = OrderedDict()

# model performance in FE
for label in LABELS:
    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    train_loader, test_loader = get_dataloaders(
        dataset="CIFAR10",
        batch_size=1000,
        pseudo_labels=False,
        normalize=True,
        train_shuffle=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__te_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }



/data/weijing/DI-ting_normalization/models/CIFAR10/model_data-normalization/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:30<00:00,  1.62it/s]


0.9999
0.9385
/data/weijing/DI-ting_normalization/models/CIFAR10/model_min-max--1-and-1/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.58it/s]


0.7944
0.7617
/data/weijing/DI-ting_normalization/models/CIFAR10/model_normalization-without-mean/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


...

0.6434
0.6185
/data/weijing/DI-ting_normalization/models/CIFAR10/model_normalization-without-std/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.58it/s]


0.29286
0.2935
/data/weijing/DI-ting_normalization/models/CIFAR10/model_rgb-grayscale/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.58it/s]


0.27302
0.2727


In [9]:
pprint(model_name__te_acc)

# Serializing json
json_object = json.dumps(model_name__te_acc, indent=2)
 
# Writing to sample.json
with open("fe_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model_data-normalization',
              {'test_acc': 0.9385, 'train_acc': 0.9999}),
             ('model_min-max--1-and-1',
              {'test_acc': 0.7617, 'train_acc': 0.7944}),
             ('model_normalization-without-mean',
              {'test_acc': 0.6185, 'train_acc': 0.6434}),
             ('model_normalization-without-std',
              {'test_acc': 0.2935, 'train_acc': 0.29286}),
             ('model_rgb-grayscale',
              {'test_acc': 0.2727, 'train_acc': 0.27302})])
